# Easily Analyze ground state wave function

In [1]:
from molbench.orchestrator import Orchestrator
from molbench.utils.omegaconf import load_molecule_cfg
from molbench.wfn import wf_overlap

## 1. Load Molecule Setting from Configure File

In [2]:
def initialize_molecule(molecule_name: str, overrides: dict | None = None):
    config = load_molecule_cfg(molecule_name, overrides=overrides)
    executor = Orchestrator(
        geometry=config.molecule.geometry,
        basis=config.molecule.basis,
        nelecas=config.molecule.nelecas,
        norbcas=config.molecule.norbcas,
        spin=config.molecule.spin,
        charge=config.molecule.charge,
    )
    return config, executor

In [3]:
molecule_name = "N2"

## 2. Compare the ground states at equilibrium and dissociation limit

Here we compute the CASCI ground-state wavefunction at an equilibrium bond length and at a stretched (dissociation-limit) bond length, then print the leading determinants.

From the printed coefficients, you can verify how the character of the ground state changes: near equilibrium it is largely single-reference (one dominant determinant), while at dissociation the weight spreads across multiple determinants, indicating stronger static correlation.

In [4]:
# Equilibrium
eq_bond_length = 1.1
eq_config, eq_executor = initialize_molecule(molecule_name, overrides={"distance": eq_bond_length})
eq_ground_state = eq_executor.do_casci().wfn
eq_ground_state.print_topdets(n=10)

1234567890  |  1234567890  |  coeff
----------------------------------------
1111111000  |  1111111000  |  0.957506
1111101010  |  1111101010  |  -0.116304
1111011100  |  1111011100  |  -0.116304
1111101010  |  1111011100  |  0.068840
1111011100  |  1111101010  |  0.068840
1110111100  |  1110111100  |  -0.054709
1110111010  |  1110111010  |  -0.054709
1111111000  |  1111001110  |  -0.049056
1111001110  |  1111111000  |  -0.049056
1111101100  |  1111011100  |  0.039404


In [5]:
# Dissociation limit
diss_bond_length = 3.0
diss_config, diss_executor = initialize_molecule(molecule_name, overrides={"distance": diss_bond_length})
diss_ground_state = diss_executor.do_casci().wfn
diss_ground_state.print_topdets(n=10)

1234567890  |  1234567890  |  coeff
----------------------------------------
1111111000  |  1111111000  |  0.267535
1111110010  |  1111110010  |  -0.263597
1111101100  |  1111101100  |  -0.263597
1111100110  |  1111100110  |  0.259786
1111011001  |  1111011001  |  -0.239826
1111001101  |  1111001101  |  0.236803
1111010011  |  1111010011  |  0.236803
1111000111  |  1111000111  |  -0.233880
1111110100  |  1111101010  |  0.175717
1111101010  |  1111110100  |  0.175710


## 3. Compare Single and Multi reference algotithm

In this section we treat the CASCI ground state as a reference and compare how well different methods reproduce it by computing wavefunction overlaps: single-reference methods (HF/CISD/CCSD) versus a multi-reference method (DMRG).

From the overlap values, you can confirm which methods remain close to the CASCI reference at equilibrium and which degrade at dissociation; in particular, multi-reference DMRG typically tracks the strongly correlated (stretched-bond) regime better than single-reference approaches.

In [6]:
# single Reference algorithm
hf_eq_wfn = eq_executor.do_hf().wfn
hf_diss_wfn = diss_executor.do_hf().wfn
ccsd_eq_wfn = eq_executor.do_ccsd().wfn
ccsd_diss_wfn = diss_executor.do_ccsd().wfn
cisd_eq_wfn = eq_executor.do_cisd().wfn
cisd_diss_wfn = diss_executor.do_cisd().wfn


# Multi Reference algorithm
schedule = [
    [20, 40, 60],          # bond dimensions
    [5, 5, 5],             # n_sweeps at each bond dim
    [1e-4, 1e-5, 0.0],     # noise values
    [1e-6, 1e-8, 1e-10],   # Davidson thresholds
]
dmrg_eq_wfn = eq_executor.do_dmrg(schedule=schedule, verbose=0).wfn
dmrg_diss_wfn = diss_executor.do_dmrg(schedule=schedule, verbose=0).wfn


In [7]:
# Calculate overlap
def print_overlap(label, wfn_ref, wfn_cmp):
    ov = wf_overlap(wfn_ref, wfn_cmp)
    print(f"{label}: {ov:.6f}")

print("=== Equilibrium (R = {:.2f}) Overlaps ===".format(eq_bond_length))
print_overlap("HF (Single)", hf_eq_wfn, eq_ground_state)
print_overlap("CISD (Single)", cisd_eq_wfn, eq_ground_state)
print_overlap("CCSD (Single)", ccsd_eq_wfn, eq_ground_state)
print_overlap("DMRG (Multi)", dmrg_eq_wfn, eq_ground_state)
print()

print("=== Dissociation limit (R = {:.2f}) Overlaps ===".format(diss_bond_length))
print_overlap("HF (Single)", hf_diss_wfn, diss_ground_state)
print_overlap("CISD (Single)", cisd_diss_wfn, diss_ground_state)
print_overlap("CCSD (Single)", ccsd_diss_wfn, diss_ground_state)
print_overlap("DMRG (Multi)", dmrg_diss_wfn, diss_ground_state)

=== Equilibrium (R = 1.10) Overlaps ===
HF (Single): 0.957506
CISD (Single): 0.997635
CCSD (Single): 0.997838
DMRG (Multi): 0.999939

=== Dissociation limit (R = 3.00) Overlaps ===
HF (Single): 0.267535
CISD (Single): 0.714019
CCSD (Single): 0.558476
DMRG (Multi): 1.000000
